#Sistema de Recuperación de Información Booleano

## Actividades a realizar

1.   Cagar conjunto de datos (archivo csv)
2.   Leer el libro con la biblioteca `pandas`
3.   Tokenizar
   * Implementar método para normalizar consultas y documentos
   *  Implementar posting list con índice invertido
4.   Implementar operadores:
   * OR 
   * AND 
   * AND NOT
   * AND MÚLTIPLE
5.   Implementar SRI Booleano
     * Realizar pruebas con los operadores

## 1. Cargar conjunto de datos

In [1]:
import os

# url raw del conjunto de datos en el repositorio
url = 'https://raw.githubusercontent.com/orlandxrf/curso-pln/main/data/news_pandemia_covid19.csv'

# nombre de la carpeta para almacenar el conjunto de datos
folder = 'dataset'

# crear carpeta para almacenar el conjunto de datos
! mkdir {folder}

# renombrar conjunto de datos
name = 'news_pandemia_covid19.csv'

# establecer ruta del folder y el nombre del conjunto de datos
path = os.path.join(folder, name)

# descargar conjunto de datos y alamcenar
! wget -nc {url} -O {path}

# comprobrar
! ls -lh dataset/*

--2022-11-17 09:37:06--  https://raw.githubusercontent.com/orlandxrf/curso-pln/main/data/news_pandemia_covid19.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2554752 (2.4M) [text/plain]
Saving to: ‘dataset/news_pandemia_covid19.csv’

dataset/news_pandem 100%[===================>]   2.44M  --.-KB/s    in 0.04s   

2022-11-17 09:37:07 (56.7 MB/s) - ‘dataset/news_pandemia_covid19.csv’ saved [2554752/2554752]

-rw-r--r-- 1 root root 2.5M Nov 17 09:37 dataset/news_pandemia_covid19.csv


## 2. Leer conjunto de datos

In [2]:
# leer el conjunto de datos

import pandas as pd
import numpy as np

path = 'dataset/news_pandemia_covid19.csv'

df = pd.read_csv(path, sep=';')
df.head(5)

# extraer solo la columna Contenido
contenido2 = df['Contenido']
contenido2 = contenido2[contenido2.notnull()] # eliminar los campos vacíos de la columna Contenido

noticias = contenido2.values.tolist()
print(noticias[0])

El programa "The View" regresa a transmitir de manera remota, luego de que Whoopi Golberg, una de sus copresentadoras, saliera positiva por covid-19. Para evitar más contagios dentro de la producción, las presentadoras estarán produciendo el programa desde sus casas.


#### Observamos algunos elementos de la lista **noticias**

In [ ]:
noticias[:3]

['El programa "The View" regresa a transmitir de manera remota, luego de que Whoopi Golberg, una de sus copresentadoras, saliera positiva por covid-19. Para evitar más contagios dentro de la producción, las presentadoras estarán produciendo el programa desde sus casas.',
 'Andrés Oppenheimer analiza y opina en su columna sobre la nueva ola de la pandemia, los rebrotes de covid-19 y la variante ómicron. También le pregunta al Dr. Vivek Murthy, director general de Sanidad de EE.UU., sobre la variante ómicron y las nuevas pastillas contra el covid-19. Este domingo en "Oppenheimer Presenta" a las 8 P.M., hora de Miami.',
 '(CNN) –– Es posible que el covid-19 no haya terminado con nosotros. Pero nosotros sí que estamos cansados de él.Ese sentimiento lo grita a voz en cuello una nueva encuesta nacional de la Universidad de Monmouth que se publicó esta semana.Consideremos:1. Más de 3 de cada 4 estadounidenses (el 77%) apoyan la flexibilización de las pautas de los Centros para el Control y la

## 3. Tokenizar

* Limpiar el conjunto de datos de signos de puntuación
* Los caracteres permitidos serán letras y numeros
* Convertir a minúsculas
* Eliminar las palabras vacías (en español e inglés)



In [3]:
from spacy.lang.es.stop_words import STOP_WORDS as ES_STOP_WORDS
from spacy.lang.en.stop_words import STOP_WORDS as EN_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer
import re

# unir los dos conjuntos de palabras vacías
BILINGUAL_STOP_WORDS = ES_STOP_WORDS.union(EN_STOP_WORDS)

#### Observamos cómo quedó el set **BILINGUAL_STOP_WORDS**

In [ ]:
BILINGUAL_STOP_WORDS

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'acuerdo',
 'adelante',
 'ademas',
 'además',
 'afirmó',
 'after',
 'afterwards',
 'again',
 'against',
 'agregó',
 'ahi',
 'ahora',
 'ahí',
 'al',
 'algo',
 'alguna',
 'algunas',
 'alguno',
 'algunos',
 'algún',
 'all',
 'alli',
 'allí',
 'almost',
 'alone',
 'along',
 'already',
 'alrededor',
 'also',
 'although',
 'always',
 'am',
 'ambos',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'ante',
 'anterior',
 'antes',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'apenas',
 'aproximadamente',
 'aquel',
 'aquella',
 'aquellas',
 'aquello',
 'aquellos',
 'aqui',
 'aquél',
 'aquélla',
 'aquéllas',
 'aquéllos',
 'aquí',
 'are',
 'around',
 'arriba',
 'as',
 'aseguró',
 'asi',
 'así',
 'at',
 'atras',
 'aun',
 'aunque',
 'añadió',
 'aún',
 'back',
 'bajo',
 'bastante',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'b

####Creando una función para normalizar y limpiar los documentos

In [4]:
def normalizer(doc):
  doc_temp = re.sub(u'[^0-9a-zA-ZáéíóúÁÉÍÓÚñÑü]', ' ', doc) #reemplazamos todo lo que no sea alfanumérico por " " en el documento
  doc_temp = doc_temp.lower() #convertimos a minúscula el documento
  doc_temp = " ".join(doc_temp.split()) #quitamos espacios duplicados

#quitamos del documento las stop words (tanto en inglés como en español)
  words_list = []

  for word in doc_temp.split():
    if word not in BILINGUAL_STOP_WORDS:
      words_list.append(word)

  doc_temp = " ".join(words_list) #reconstituimos el documento
  return doc_temp #devolvemos el documento

####Probamos la función recién creada para normalizar documentos:

In [5]:
normalizer('El programa "The View" regresa a transmitir de manera remota, luego de que Whoopi Golberg, una de sus copresentadoras, saliera positiva por covid-19.')

'programa view regresa transmitir remota whoopi golberg copresentadoras saliera positiva covid 19'

#### Instanciamos spacy (la parte en español)

In [6]:
#Instalamos lo adicional de spacy
!pip install -U spacy
!python -m spacy download es_core_news_sm

#Importamos
import spacy

#E instanciamos
nlp = spacy.load("es_core_news_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 6.8 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.2
    Uninstalling spacy-3.4.2:
      Successfully uninstalled spacy-3.4.2


2022-11-17 09:42:21.101265: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 1.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


##### Normalizo cada noticia y la agrego a la lista "noticias_norm" y armo un diccionario con el vocabulario (como key) y una lista vacía como llave (en la que pondré las posting lists)


In [7]:
#[normalizer(noticia) for noticia in noticias ]
noticias_norm = []
vocabulario = {}
for noticia in noticias:
  noticia_norm = normalizer(noticia)
  noticias_norm.append(noticia_norm)

  vocabulario.update( {token.text:[] for token in nlp(noticia_norm)} )
#numeros = [num for num in range(11)]
#print(numeros)

#### Imprimo las primeras posiciones de mi lista de noticias normalizadas

In [ ]:
noticias_norm[:4]

['programa view regresa transmitir remota whoopi golberg copresentadoras saliera positiva covid 19 evitar contagios producción presentadoras estarán produciendo programa casas',
 'andrés oppenheimer analiza opina columna ola pandemia rebrotes covid 19 variante ómicron pregunta dr vivek murthy director general sanidad ee uu variante ómicron pastillas covid 19 domingo oppenheimer presenta 8 p m hora miami',
 'cnn covid 19 terminado cansados sentimiento grita voz cuello encuesta nacional universidad monmouth publicó semana consideremos 1 3 4 estadounidenses 77 apoyan flexibilización pautas centros control prevención enfermedades cdc siglas inglés mascarillas distanciamiento social áreas bajas tasas covid 19 2 3 4 73 afirmación hora aceptemos covid 19 quedarse simplemente seguir vidas 3 grupo 4 10 quieren establezcan futuras regulaciones mandatos relacionados covid 19 encuesta ap norc 44 estadounidenses frecuencia máscaras cerca personas hogares 65 enero 2022 82 febrero 2021 cifras llegan 

Imprimo también mi diccionario donde armaré los posting lists

In [ ]:
vocabulario

{'programa': [],
 'view': [],
 'regresa': [],
 'transmitir': [],
 'remota': [],
 'whoopi': [],
 'golberg': [],
 'copresentadoras': [],
 'saliera': [],
 'positiva': [],
 'covid': [],
 '19': [],
 'evitar': [],
 'contagios': [],
 'producción': [],
 'presentadoras': [],
 'estarán': [],
 'produciendo': [],
 'casas': [],
 'andrés': [],
 'oppenheimer': [],
 'analiza': [],
 'opina': [],
 'columna': [],
 'ola': [],
 'pandemia': [],
 'rebrotes': [],
 'variante': [],
 'ómicron': [],
 'pregunta': [],
 'dr': [],
 'vivek': [],
 'murthy': [],
 'director': [],
 'general': [],
 'sanidad': [],
 'ee': [],
 'uu': [],
 'pastillas': [],
 'domingo': [],
 'presenta': [],
 '8': [],
 'p': [],
 'm': [],
 'hora': [],
 'miami': [],
 'cnn': [],
 'terminado': [],
 'cansados': [],
 'sentimiento': [],
 'grita': [],
 'voz': [],
 'cuello': [],
 'encuesta': [],
 'nacional': [],
 'universidad': [],
 'monmouth': [],
 'publicó': [],
 'semana': [],
 'consideremos': [],
 '1': [],
 '3': [],
 '4': [],
 'estadounidenses': [],
 '

In [ ]:
vocabulario["covid"]

[]

## Posting list con índice invertido

In [ ]:
for token in vocabulario.keys():
  for index, noticia in enumerate(noticias_norm):
    if token in noticia:
      if index not in vocabulario[token]:
        vocabulario[token].append(index)

####Imprimo el diccionario donde para cada key (o token) ya tengo su posting list

In [ ]:
print(vocabulario)

{'programa': [0, 19, 30, 32, 74, 83, 108, 127, 143, 146, 150, 159, 164, 182, 188, 189, 194, 205, 210, 214, 222, 227, 231, 240, 245, 252, 261, 262, 265, 268, 289, 290, 297, 310, 313, 317, 328, 329, 334, 344, 345, 350, 360, 373, 381, 387, 417, 418, 425, 426, 427, 431, 449, 458, 459, 460, 492, 512, 524, 526, 527, 537, 545, 550, 563, 577, 586, 591, 592, 594, 601, 602, 605, 612, 619, 625, 634, 635, 636, 638, 641, 642, 652, 662, 668, 677, 685, 687, 693, 696, 699, 712, 729, 734, 735, 747, 751, 759, 763, 773, 774, 777, 779, 781, 788, 798, 810, 828, 830, 831, 832, 834, 835, 840, 842, 843, 845, 848, 855, 857, 861, 863, 865, 867, 873, 878, 888, 897, 904, 906, 909, 910, 911, 919, 920, 922, 924, 928, 945, 947, 949, 954, 956, 957, 968], 'view': [0, 446], 'regresa': [0, 26, 55, 56, 88, 107, 145, 170, 182, 262, 280, 290, 326, 423, 444, 446, 453, 466, 480, 491, 513, 520, 528, 546, 563, 566, 572, 584, 593, 615, 661, 776, 850, 893, 913], 'transmitir': [0, 36, 107, 145, 357, 372, 415, 417, 430, 446, 457, 

## 4. Implementar operadores Binarios

Implementar los operadores `OR`, `AND` y `AND NOT` para consultas binarias

> **Nota**: El vector resultante de las operaciones binarias debe estar ordenado en forma ascedente y sin repeticiones.



### Operador OR

*  El operador `OR` une (UNION) los `posting list` de los documentos donde aparecen el termino 1 y el termino 2.

<table>
<tbody>
<tr>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/Operador_OR.png" alt="Operador OR"/></td>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/or.png" alt="Operación OR" width=600/></td>
</tr>
</tbody>
</table>

Se recuperan los documentos de:

1. Antony and Cleopatra
2. Julios Caesar
3. Hamlet
4. Othello
5. Macbeth

Crear una función de Python para realizar la operación con el operador `OR`.
Ejemplo:

```python
# dadas las listas:
lista1 = [803, 1025, 2645, 5789, 6478, 6772]
lista2 = [60, 524, 2645]

# realizar la operación OR
lista_resultado = operador_OR(lista1, lista2)

# resultado
lista_resultado = [60, 524, 803, 1025, 2645, 5789, 6478, 6772]
```

##### La siguiente solo es sección de pruebas:

In [ ]:
# dadas las listas:
lista1 = [60, 803, 1025, 2645, 5789, 6478, 6772,6772]
lista2 = [60, 524, 2645]

In [ ]:
# convierto las listas a sets:
lista1 = set(lista1)
lista2 = set(lista2)
print(lista1)
print(lista2)

{1025, 803, 6478, 6772, 2645, 60, 5789}
{60, 524, 2645}


In [ ]:
#Prueba para la unión de dos sets
lista1.union(lista2)

{60, 524, 803, 1025, 2645, 5789, 6478, 6772}

In [ ]:
#Prueba para la intersección de dos sets
lista1.intersection(lista2)

{60, 2645}

In [ ]:
word1_plist = set(vocabulario["transmitir"])

In [ ]:
word2_plist = set(vocabulario["contagiar"])

In [ ]:
print(word1_plist.union(word2_plist))

{0, 641, 262, 520, 264, 139, 653, 145, 401, 531, 274, 538, 26, 668, 539, 415, 544, 417, 546, 675, 36, 550, 552, 810, 299, 557, 430, 562, 434, 563, 566, 572, 446, 576, 320, 66, 580, 198, 839, 582, 457, 458, 587, 716, 461, 464, 593, 467, 469, 473, 480, 357, 107, 365, 494, 368, 624, 498, 372, 501}


####Implementamos una función para el operador binario OR

In [ ]:
# implementar función para el operador binario OR

def operador_OR(word1, word2):
  word1_plist = set(vocabulario[word1])
  word2_plist = set(vocabulario[word2])
  return word1_plist.union(word2_plist)

# realizar la operación OR
#lista_resultado = operador_OR(lista1, lista2)

# resultado
#lista_resultado = [60, 524, 803, 1025, 2645, 5789, 6478, 6772]

##### Y probamos:

In [ ]:
print(operador_OR("transmitir", "contagiar"))

{0, 641, 262, 520, 264, 139, 653, 145, 401, 531, 274, 538, 26, 668, 539, 415, 544, 417, 546, 675, 36, 550, 552, 810, 299, 557, 430, 562, 434, 563, 566, 572, 446, 576, 320, 66, 580, 198, 839, 582, 457, 458, 587, 716, 461, 464, 593, 467, 469, 473, 480, 357, 107, 365, 494, 368, 624, 498, 372, 501}


### Operador AND

*  El operador `AND` intersecta (INTERSECTION) los `posting list` donde aparece el termino 1 y el termino 2.

<table>
<tbody>
<tr>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/Operador_AND.png" alt="Operador AND"/></td>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/and.png" alt="Operación AND" width=600/></td>
</tr>
</tbody>
</table>

Se recuperan los documentos de:

1. Antony and Cleopatra
2. Julios Caesar
3. Hamlet


Crear una función de Python para realizar la operación con el operador `AND`.
Ejemplo:

```python
# dadas las listas:
lista1 = [803, 1025, 2645, 5789, 6478, 6772]
lista2 = [60, 524, 2645]

# realizar la operación AND
lista_resultado = operador_AND(lista1, lista2)

# resultado
lista_resultado = [2645]
```

####Implementamos una función para el operador binario AND

In [ ]:
# implementar función para el operador binario AND

def operador_AND(word1, word2):
  word1_plist = set(vocabulario[word1])
  word2_plist = set(vocabulario[word2])
  return word1_plist.intersection(word2_plist)

##### Y probamos:

In [ ]:
operador_AND("covid", "obrador")

{12, 99, 326, 408, 609, 660, 710}

### Operador AND NOT

*  Operador `AND NOT` compara los elementos de ambos `posting list`, descartando los elementos del `posting list` del termino 2 que aparecen en el `posting list` del termino 1.

<table>
<tbody>
<tr>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/Operador_AND_NOT.png" alt="Operador AND NOT"/></td>
<td><img src="https://raw.githubusercontent.com/orlandxrf/curso-pln/main/img/and_not.png" alt="Operación AND NOT" width=600/></td>
</tr>
</tbody>
</table>

Se recuperan los documentos de:

1. Antony and Cleopatra
3. Hamlet

Crear una función de Python para realizar la operación con el operador `AND NOT`.
Ejemplo:

```python
# dadas las listas:
lista1 = [60, 803, 1025, 2645, 5789, 6478, 6772]
lista2 = [60, 524, 2645, 5789, 6478, 6772]

print ( op_AND_NOT(lista1, lista2) )

# realizar la operación AND NOT
lista_resultado = operador_AND_NOT(lista1, lista2)

# resultado
lista_resultado = [803, 1025]
```

####Implementamos una función para el operador binario AND NOT

In [ ]:
# implementar función para el operador binario AND NOT
def operador_AND_NOT(word1, word2):
  word1_plist = set(vocabulario[word1])
  word2_plist = set(vocabulario[word2])
  return word1_plist.difference(word2_plist)

##### Y probamos:

In [ ]:
operador_AND_NOT("transmitir", "positivo")

{0,
 145,
 372,
 415,
 430,
 446,
 457,
 464,
 467,
 473,
 480,
 501,
 531,
 550,
 557,
 566,
 580,
 587,
 593,
 668,
 675,
 810,
 839}

### Operador AND MÚLTIPLE

Este operador obtiene todos los documentos intersectados de los documentos donde aparecen sus terminos correspondientes.

Crear una función de Python para realizar las operaciones con el operador `AND MULTIPLE`. Ejemplo:

```python
lista_documentos = [
  [60, 803, 1025, 2645, 5789, 6478, 6772], # lista 1
  [60, 524, 2645, 5789, 6478, 6772],       # lista 2
  [1, 25, 60, 358, 6478],                  # lista 3
]

resultado = op_Multiple_AND(lista_documentos)
print (resultado)
# [60, 6478]
```




####Implementamos una función para el operador múltiple AND

In [ ]:
from pandas.core import strings
# implementar el operador múltiple AND

def op_Multiple_AND(string_list):
  acum_list = ()
  for word in string_list:
    # Hacemos la intersección de set(vocabulario[word]) con acum_list
    if len(acum_list) == 0:
      acum_list = set(vocabulario[word])
    # Asignamos la intersección a acum_list
    else:
      acum_list = acum_list.intersection(set(vocabulario[word]))
  return acum_list


##### Y probamos:

In [ ]:
result = op_Multiple_AND(["transmitir", "positivo", "enero"])
print(result)

{576, 417}


In [ ]:
print(noticias_norm[576])
print(noticias_norm[417])

cnn evidencia animales jueguen papel importante propagación coronavirus humanos tomar precauciones ayudar mantener personas mascotas lunes funcionarios centros control prevención enfermedades ee uu cdc información limitada disponible fecha riesgo animales incluidas mascotas transmitan covid 19 personas dr casey barton behravesh funcionario cdc sesión informativa lunes evidencia sugiere covid 19 probablemente originó animales generalizarse humanos mediados enero conocimiento 187 animales 22 países infección confirmada sars cov 2 behravesh números incluyen visones granjas relacionado muertes animales virus cdc siguiendo cerca investigación infecciones coronavirus animales categorizado animales riesgo infección animales altamente susceptibles virus incluyen gatos hámsteres primates humanos conejos visones ciervos behravesh fotos leopardos visones tigres animales positivo coronavirusbehravesh animales compañía especialmente gatos perros grupo principal especies animales afectadas coronavir

## 5. Sistema de Recuperación de Información

Implementar el sistema de recuperación de información booleano empleando los índices invertidos y los operadores OR, AND y NOT AND.

1.   Solicitar consulta a buscar con `input` de `Python`
2.   Normalizar la consulta empleando la función previamente implementada para normalizar documentos
3.   Verificar que al menos uno de los terminos exista en nuestra estructura de índice invertido
4.   Obtener las listas de documentos por cada termino (2 terminos) y aplicar alguno de los operadores implementados: `OR`, `AND` o `AND NOT` a las listas de documentos obtenidos.
5.   Para consultas con múltiples terminos, obtener las listas de documentos de cada termino y aplicar el operador `AND MULTIPLE`.
6.    De los resultados obtenidos en 4 y 5 mostrar los primeros 5 documentos originales (sin normalizar).



In [ ]:
query = input("Escribe tu consulta, please! ")

splitted_query = query.split(" ")
if query == "":
  print("Por favor ingresa una consulta")
elif len(splitted_query) == 1:
  print("Por favor incluye en tu consulta al menos dos palabras")
elif len(splitted_query) >= 2:
  #Normalizamos
  norm_query = normalizer(query)
  spl_norm_query = norm_query.split(" ")
  spl_norm_query_ok = []
  for word in spl_norm_query:
    if word in vocabulario:
      spl_norm_query_ok.append(word)
  if len(spl_norm_query_ok) == 0:
    print(f'Tu consulta "{query}" no fue hallada en ninguna noticia')
  if len(spl_norm_query_ok) == 1:
    print(f'Tu consulta "{query}" fue hallada en las noticias: {vocabulario[spl_norm_query_ok[0]]}')
  if len(spl_norm_query_ok) == 2:
    #Aplicamos los operadores OR, AND, y AND NOT:
    print(f'Tu consulta "{query}" fue hallada en las noticias (usando el operador OR): {operador_OR(spl_norm_query_ok[0],spl_norm_query_ok[1])}')
    print(f'Tu consulta "{query}" fue hallada en las noticias (usando el operador AND): {operador_AND(spl_norm_query_ok[0],spl_norm_query_ok[1])}')
    print(f'Tu consulta "{query}" fue hallada en las noticias (usando el operador AND NOT): {operador_AND_NOT(spl_norm_query_ok[0],spl_norm_query_ok[1])}')
  if len(spl_norm_query_ok) > 2:
    #Aplicamos el operador AND MÚLTIPLE:
    print(f'Tu consulta "{query}" fue hallada en las noticias (usando el operador MULTIPLE AND): {op_Multiple_AND(spl_norm_query_ok)}')


Escribe tu consulta, please! covid abril positivo
Tu consulta "covid abril positivo" fue hallada en las noticias (usando el operador MULTIPLE AND): {280, 546, 67, 578, 227, 262, 326, 357, 489, 74, 537, 271, 562, 536, 665, 444, 127}
